In [ ]:
from selenium import webdriver
import re
import pandas as pd
import time

# 전체 동영상 url 가져오기

In [ ]:
url = 'https://tv.naver.com/tvchosun.mistertrot/clips'
driver = webdriver.Chrome('../chromedriver.exe')
driver.implicitly_wait(10) # 페이지가 로딩되는데 10초까지 기다린다 -> 그 전에 로딩이 완료되면 바로 다음 코드를 수행한다
# 전역 변수로 한 번만 선언하면 이후에 자동 적용됨
driver.get(url)

video_url = [] # 각 동영상 url 저장할 리스트

last_height = driver.execute_script("return document.body.scrollHeight")      
cnt = 0 # while 문 탈출용

while cnt < 1:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 창 높이까지 스크롤바 내리기 
    time.sleep(1)
    elements = driver.find_element_by_xpath('//*[@id="cds_flick"]/div/div[3]/div/div/div/div[2]/div[3]/a')
    driver.execute_script("arguments[0].click();", elements) # "아래 화살표" 버튼 클릭 명령(자바스크립트 명령어)
    # (=driver.send_keys('\n')) -> 해당 링크, 명령어에 엔터를 치는 것
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") # 맨 마지막까지 스크롤하면 바로 아래 리스트가
    # 뜨지 않는 경우가 있는데, 창을 조금 올려서 리스트가 로딩될 수 있도록 함 
    time.sleep(2)
    
    whole = driver.find_element_by_xpath('//*[@id="cds_flick"]/div/div[3]/div/div/div/div[2]/div[2]')
    video_lst = whole.find_elements_by_tag_name('dt > a')
    
    for video in video_lst:
        video_url.append(video.get_attribute('href'))
        
    cnt += 1
    
driver.quit()

# 각 동영상의 댓글 정보 가져오기 
* 댓글 내용
* 댓글 좋아요 수
* 댓글 싫어요 수
* 댓글 입력 시간

In [ ]:
# 네이버 tv 웹페이지
# video_url에 있는 모든 동영상을 실현하고 싶은 경우 for문 사용
"""
driver = webdriver.Chrome('../chromedriver.exe')
driver.implicitly_wait(10) 
for url in video_url:
    driver.get(url)
    아래 코드와 동일
"""

# 샘플 테스트
url = 'https://tv.naver.com/v/12711849/list/571830'
driver = webdriver.Chrome('../chromedriver.exe')
driver.implicitly_wait(10) 
driver.get(url)

# 첫 페이지 댓글 
content_lst1 = []
comments = driver.find_elements_by_xpath('//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
body = comments[0].find_elements_by_class_name('u_cbox_text_wrap') # 실제 댓글 내용
reaction = comments[0].find_elements_by_class_name('u_cbox_recomm_set') # 좋아요, 싫어요 모음
date = comments[0].find_elements_by_class_name('u_cbox_info_base') # 댓글 작성 시간

for item in zip(body, reaction, date):
    content_lst1.append(
    [
        item[0].text,
        item[1].text.split('\n')[2],
        item[1].text.split('\n')[4],
        item[2].text.split('\n')[0]
    ])
    
# 2페이지로 넘어가고 댓글 수집
content_lst2 = []
elements = driver.find_element_by_xpath('//*[@id="cbox_module"]/div/div[8]/div/a[1]')
driver.execute_script("arguments[0].click();", elements)
time.sleep(2)

comments = driver.find_elements_by_xpath('//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
body = comments[0].find_elements_by_class_name('u_cbox_text_wrap')
reaction = comments[0].find_elements_by_class_name('u_cbox_recomm_set')
date = comments[0].find_elements_by_class_name('u_cbox_info_base')

for item in zip(body, reaction, date):
    content_lst2.append(
    [
        item[0].text,
        item[1].text.split('\n')[2],
        item[1].text.split('\n')[4],
        item[2].text.split('\n')[0]
    ])

# 3~6페이지 댓글
content_lst3 = []
for l in range(3, 6+1):
    elements = driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[8]/div/a[{l}]')
    driver.execute_script("arguments[0].click();", elements)
    time.sleep(2)
    
    comments = driver.find_elements_by_xpath('//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
    body = comments[0].find_elements_by_class_name('u_cbox_text_wrap')
    reaction = comments[0].find_elements_by_class_name('u_cbox_recomm_set')
    date = comments[0].find_elements_by_class_name('u_cbox_info_base')
    
    for item in zip(body, reaction, date):
        content_lst3.append(
        [
            item[0].text,
            item[1].text.split('\n')[2],
            item[1].text.split('\n')[4],
            item[2].text.split('\n')[0]
        ])

# 7페이지 ~ 끝까지  
content_lst4 = []
while True:
    for i in range(3, 7+1):
        elements = driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[8]/div/a[{i}]')
        driver.execute_script("arguments[0].click();", elements)
        time.sleep(3)

    comments = driver.find_elements_by_xpath('//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
    body = comments[0].find_elements_by_class_name('u_cbox_text_wrap')
    reaction = comments[0].find_elements_by_class_name('u_cbox_recomm_set')
    date = comments[0].find_elements_by_class_name('u_cbox_info_base')

    for item in zip(body, reaction, date):
        content_lst4.append(
        [
            item[0].text,
            item[1].text.split('\n')[2],
            item[1].text.split('\n')[4],
            item[2].text.split('\n')[0]
        ])

# 4가지 리스트 합쳐주기 
content_lst = content_lst1 + content_lst2 + content_lst3 + content_lst4
# 데이터프레임 생성
content_info = pd.DataFrame(content_lst, columns=['comment', 'likes', 'dislikes', 'date'])
driver.quit()